In [148]:
import json
import re

from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.core import Annotation, Segment

In [149]:
# Calculates accurateness of Whisper ASR

# Normalizes the text by removing punctuation and converting to lowercase.
def normalize_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = text.lower()
    return text


# Calculates the Word Error Rate (WER).
def wer(reference, hypothesis):
    ref_words = normalize_text(reference).split()
    hyp_words = normalize_text(hypothesis).split()

    # Initializes matrix
    d = [[0] * (len(hyp_words) + 1) for _ in range(len(ref_words) + 1)]

    # Fills the first row and column of the matrix
    for i in range(len(ref_words) + 1):
        d[i][0] = i
    for j in range(len(hyp_words) + 1):
        d[0][j] = j

    # Populates matrix
    for i in range(1, len(ref_words) + 1):
        for j in range(1, len(hyp_words) + 1):
            if ref_words[i - 1] == hyp_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                substitution = d[i - 1][j - 1] + 1
                insertion = d[i][j - 1] + 1
                deletion = d[i - 1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    # The WER is the number of edits divided by the number of words in the reference
    wer_value = d[len(ref_words)][len(hyp_words)] / len(ref_words)

    return wer_value

In [167]:
# Haarlem - 2016 - 246556.mp4 - 36:07
ref = "Als er iemand stemming vraagt overigens. Als niemand stemming vraagt. Ja, wel vanuit de SP wordt dat gevraagd. Wie in de regio? Dat is dat gevraagd. Wie is voor dit beleidskader? Nou had ik beter eerst kunnen vragen wie tegen was. 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20. Ik zie de SP die stemt niet voor. Even kijken. Ja, de rest wel. Dus met stemmen van de SP tegen is het aangenomen. Dan gaan wij naar agenda punt 9. Dat is het on-eigelijk gebruik van gemeentegrond. Ik heb begrepen dat er wat punten zijn aangepast na de behandeling in de commissie. Maar dat de wethouder nog even een toevoeging heeft op die aanpassingen. De heer Bottigen. Meneer de voorzitter. Er is in de stukken een fout geslopen in het collegebesluit. Wat als raadsbesluit staat. Daar wordt wel gesproken over dat het gaat over 2,2 miljoen verdeeld over 10 jaar. Maar dat dat per jaar wordt vastgesteld. Maar in het raadsbesluit is dat verkeerd opgenomen. Dus de tekst van het stuk van het raadsbesluit. Het is zo dat eigenlijk daarmee die motie die is ingediend op dit vlak. Of het abonnement. Waarbij staat dat er gedurende het bedrag van de kadernoten van vier jaar. En dat dan per jaar wordt gekeken of dat bedrag ingezet kan worden. Dat is ook de formulering zoals het college die heeft gedaan. Alleen die is in het raadsvoorstel onvoldoende of niet verwerkt. Dus we zijn het eens met elkaar. Waarbij wel de kanttekening overigens dat het geen zin heeft om bijvoorbeeld volgend jaar al te zeggen. Van we hebben het een jaartje geprobeerd en het werkt niet"
hyp = "Als er iemand stemming vraagt overigens. Als niemand stemming vraagt - Ja Wel vanuit de SP wordt dat gevraagd. Wie is voor dit (beleidskader)? Ja... Nou had ik beter eerst kunnen vragen wie tegen was geloof ik. 4 5 6 7 8 9 10 11 12 13 14 15 16 17. Ik zie de SP die stemt niet voor en even kijken ja de rest wel dus dat is allen met stemmen van de SP tegen is dat aangenomen. Dan gaan wij naar agenda punt 9. Dat is het oneigenlijk gebruik van gemeente grond. Ehm ik heb begrepen dat er wat punten zijn aangepast na de behandleing in de commisie maar dat de wethouder nog even een toevoeging heeft op die aapassingen. de hoor botter. Meneer de voorzitter er is in de stukken een fout geslopen in het college besluit wat  als raadsbesluit staat er wordt wel gesproken over dat het gaat over 2 komma 2 miljoen verdeeld over tien jaar maar dat dat per jaar wordt vastgesteld maar in het raadsbesluit is dat verkeerd opgenomen. Dus de tekst van het stuk van het raadsbesluit het is zo dat eigenlijk daarmee die motie die is ingediend op dit vlak of het abonnement waarbij staat dat er gedurende het bedrag nu van de kader noten van de vier jaar en dat dan per jaar wordt gekeken of dat bedrag ingezet kan worden, dat is ook de zeg maar de formulering zoals het college die heeft gedaan. Alleen die is in het raadsvoorstel onvoldoende of niet verwerkt. Dus we zijn het eens met elkaar. Waarbij wel de kanttekening overigens dat het geen zin heeft  om bijvoorbeeld volgend jaar al te zeggen van we hebben het een jaartje geprobeerd en het werkt niet."
error_rate = wer(ref, hyp)
print(f"WER: {error_rate:.2f}")

# Haarlem - 2019 - 622012.mp4 - 1:46:45
ref = "Dank u wel. De heer Visser, Christen Unie. Dank u wel, voorzitter. Wat betreft de oude stukken die opnieuw geheim verklaaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op. Zo snel mogelijk in het najaar gaan we er opnieuw over hebben. Wat betreft het PWC-rapport is niet vanaf vandaag het openbaar met uitzondering van de weggelakte delen. Dus dat is ook mooi. Wat betreft dateterrein ben ik toch een beetje benieuwd naar de reactie van het college. En ik heb inderdaad ook het gevoel dat we vanavond toch al op het zuid gaan komen. Dus waarom zouden we dat nu nog geheim verklaaren? Ik ben er niet van overtuigd wat daar het argument voor is. Dus ik zou daar eigenlijk een kort antwoord van het college op willen. Het is tot en met \u00e9\u00e9n ding niet akkoord. Al was het maar voor de symboolwerking. Maar bijlagen zeven bij de gedemd oudergeracht de uitgiftetekening. Ik heb net al gezegd ik vind het compleet een nonsens om zo'n tekening geheim te verklaren. Het college mag me wel eens uitleggen waarom dat geheim zou moeten zijn. Dank u wel. De heer van de Raad. Ja, voorzitter. Wij gaan niet akkoord met stukken die binnen 24 uur zijn ingevoerd. Dat hebben we bij het seniorenconvental aangegeven dat we daar niet meer van gediend zijn. Voor de rest heeft u gevraagd om alles in \u00e9\u00e9n keer te behandelen. U heeft ook gezegd alles komt terug. Dus u behandelt alles in een keer en het feit dat het terugkomt vinden wij dat niks geheim verklaard hoeft te worden. Dank u."
hyp = "Dank u wel. De heer visser, christen unie. Dank u wel voorzitter, wat betreft de oude stukken die opnieuw geheim verklaren kunnen wij akkoord gaan omdat het college zegt daar doen we een deadline op he, zo snel mogelijk in het najaar gaan we het er opnieuw over hebben. wat betreft pwc rapport is in ieder geval vanaf vandaag openbaar met uitzondering van de weggelakte delen. Dus dat is ook mooi. Wat betreft D-terrein  ben ik toch een beetje benieuwd naar de reactie van het college want ik heb inderdaad ook het gevoel er wordt vanavond gewoon een besluit genomen dus waarom zouden we dat nu nog geheim verklaren. Ik ben er niet van overtuigd wat daar het argument van is. Dus ik zou daar eigenlijk een kort antwoord van het college op willen. En tenslotte ga ik met 1 ding niet akkoord, al was het maar voor de symbool werking. Maar bijlage 7 bij de <> uitgifte tekening. Ik heb het net al gezegd, ik vind het complete onzin om zo'n tekening geheim te verklaren. college mag mij wel eens uitleggen waarom dat geheim zou moeten zijn. Dank u wel. de heer van de raad. ja voorzitter we gaan niet akkoord met stukken die binnen 24 uur zijn ingevoerd dat hebben we bij het senioren conferent al aangegeven dat we daar niet meer van gediend zijn. En voor de rest heeft u gevraaagd om alles in 1 keer te behandelen en u heeft ook gezegd alles komt terug, dus ik behandel alles in 1 keer, en het feit dat het terug komt vinden wij dat niks geheim verklaar hoeft te worden dank u."
error_rate = wer(ref, hyp)
print(f"WER: {error_rate:.2f}")

# Haarlem - 2024 - 1159903.mp4 - 8:10
ref = "Maar dat heeft niks te maken op zich met het referendum. Dan kan het college bevestigen dat er altijd bezwaar en beroep mogelijk is. Dat klopt niet, want in een aantal gevallen is er helemaal geen verkeersbesluit voor nodig. Iets wat niet legaal is, als je dat op een gegeven moment gaat niet meer gedoogd, dan neem je geen nieuw verkeersbesluit, want het was al illegaal. Dat moet je wel heel duidelijk markeren, want we gaan daar nu op een andere manier mee om. Ik heb al gezegd dat gebeurt alleen als er voldoende parkeerplaatsen beschikbaar zijn om de mensen met een vergunning te laten parkeren. In die situaties waarbij gelegaliseerd parkeren wordt opgeheven, daar is wel een parkeerbesluit voor en dat zal natuurlijk de gewone procedure doorlopen. Het opheffen van parkeerplaatsen, daarvoor geldt ook het reguliere participatie- en inspraakproject als dat zou gebeuren. Dan de vraag over de website. We proberen te voorkomen dat je de website vol stopt met informatie, maar nu dit zozeer een issue is"
hyp = "Maar dat heeft niks te maken op zich, met het referendum. Dan, kan het college bevestigen dat er altijd bezwaer en beroep mogelijk is. Dat klopt niet want in een aantal gevallen is er helemaal geen verkeersbesluit voor nodig. Iets wat niet legaal is, als je dat op een gegeven moment niet meer gedoogt dan neem je geen nieuw verkeersbesluit want het was al illegaal. Dan moet je natuurlijk wel heel duidelijk markeren van we gaan daar nu op een andere manier mee om en ik heb al gezegd dat gebeurt alleen als daar ook de ruimte voor is, als er dus voldoende verkeersplaatsen beschikbaar zijn om de mensen met een vergunning ook te laten parkeren. En in die situaties waarbij gelegaliseerd parkeren wordt opgeheven daar is wel een parkeer besluit voor en dat zal natuurlijk de gewone procedure doorlopen en het opheffen van parkeer plaatsen daarvoor geldt ook gewoon het reguliere participatie en inspraak project als dat zou gebeuren. dan de vraag over de website we proberen te voorkomen dat je de website volstopt met informatie maar nu dit zozeer een issue is "
error_rate = wer(ref, hyp)
print(f"WER: {error_rate:.2f}")

WER: 0.15
WER: 0.20
WER: 0.17


In [151]:
# Calculates accurateness of pyannote speaker diarisation

# Calculates the overlapping duration between two time segments.
def calculate_overlap(segment1, segment2):
    start1, end1 = segment1
    start2, end2 = segment2
    overlap_start = max(start1, start2)
    overlap_end = min(end1, end2)
    overlap = max(0, overlap_end - overlap_start)
    return overlap


# Calculates the Diarisation Error Rate (DER).
def der(ground_truth, hypothesis, start_time, end_time):
    total_time = 0
    missed_speech = 0
    false_alarm = 0
    speaker_error = 0

    for gt_start, gt_end, gt_speaker in ground_truth:
        total_time += gt_end - gt_start
        matched = False
        for hyp_start, hyp_end, hyp_speaker in hypothesis:
            overlap = calculate_overlap((gt_start, gt_end), (hyp_start, hyp_end))
            if overlap > 0:
                matched = True
                if gt_speaker != hyp_speaker:
                    speaker_error += overlap
        if not matched:
            missed_speech += gt_end - gt_start

    for hyp_start, hyp_end, hyp_speaker in hypothesis:
        matched = False
        for gt_start, gt_end, gt_speaker in ground_truth:
            overlap = calculate_overlap((gt_start, gt_end), (hyp_start, hyp_end))
            if overlap > 0:
                matched = True
        if not matched:
            false_alarm += hyp_end - hyp_start

    der = (missed_speech + false_alarm + speaker_error) / (end_time - start_time)

    return der


In [152]:
START = 3
DUR = 4
SPEAKER = 7


def round_to_nearest_interval(value, interval=0.125):
    return round(value / interval) * interval


def get_ref(filepath, start, end):
    reference = Annotation()
    with open(filepath, "r") as f:
        for line in f:
            info = line.strip().replace("\n", "").split()
            start_segm = float(info[START])
            end_segm = float(info[START]) + float(info[DUR])
            if start_segm >= start and end_segm <= end:
                reference[
                    Segment(
                        (start_segm),
                        (end_segm),
                    )
                ] = info[SPEAKER]
    # print(reference)
    # print("----")
    return reference


def get_hyp(filepath, start, end):
    data_path = "diar_data.json"
    with open(data_path, "r") as f:
        data = json.load(f)

    hypothesis = Annotation()
    for entry in data:
        if entry["file"] == filepath:
            for e in entry["data"]:
                start_segm = e[0]
                end_segm = e[1]
                if start_segm >= start and end_segm <= end:
                    hypothesis[
                        Segment(
                            (start_segm), (end_segm)
                        )
                    ] = e[2]

            # print(hypothesis)
            return hypothesis

    print(f"{filepath} not found in {data_path}")

In [153]:
def der(filepath, start, end, collar=0.0):
    diarizationErrorRate = DiarizationErrorRate(
        collar=collar,
    )
    metrics = diarizationErrorRate(
        get_ref(filepath, start, end),
        get_hyp(filepath, start, end),
        uem=Segment(start, end),
        detailed=True,
    )

    total_time = end - start
    false_alarm = metrics["false alarm"]
    missed = metrics["missed detection"]
    confusion = metrics["confusion"]
    der = metrics["diarization error rate"]

    return total_time, false_alarm, missed, confusion, der

In [164]:
filepath = "/Users/personal/Desktop/scriptie/notebooks/data/nijmegen/100000013/2022/diorizations/80ec1e6a-8c74-4f41-bcc5-6f5f27d48512.wav.rttm"
start = 3208
end = 3499
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.25), "\n")

filepath = "/Volumes/Samsung_T5/data/hoekschewaard/vergaderingen/2023/diorizations/1068492.wav.rttm"
start = 3533
end = 3798
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.25), "\n")

filepath = "/Volumes/Samsung_T5/data/ridderkerk/vergaderingen/2022/diorizations/986111.wav.rttm"
start = 3533
end = 3877
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.25), "\n")

filepath = ""
start = -1
end = -1
print(der(filepath, start, end))
print(der(filepath, start, end, collar=0.25), "\n")

(291, 4.007000000000062, 8.826999999999316, 0.3220000000001164, 0.05435173288494472)
(291, 0.6289999999989959, 5.864999999999327, 0.07200000000011642, 0.028760277002722025) 

(265, 1.829000000000633, 24.08299999999963, 3.5280000000002474, 0.13595703314414714)
(265, 0.3739999999997963, 18.496999999999844, 2.5390000000006694, 0.10526005280210118) 

(344, 27.335999999999785, 31.860999999998967, 3.5650000000000546, 0.21522433919042697)
(344, 26.35199999999986, 30.42199999999866, 3.3150000000000546, 0.2094700587739002) 



FileNotFoundError: [Errno 2] No such file or directory: ''